In [ ]:
## Gravação em formato parquet no S3 com boto3
import pandas as pd
import boto3
from io import BytesIO
import yfinance as yf
from datetime import datetime, timedelta

# 🔑 Substitua aqui pelas credenciais do usuário aws_py
ACCESS_KEY = ""
SECRET_KEY = ""
BUCKET = "fiapb3"


def save_to_parquet(df: pd.DataFrame):
    """Salva o DataFrame em formato parquet particionado por data no S3 usando boto3."""
    today = datetime.today().strftime("%Y-%m-%d")
    key = f"b3_raw/dt={today}/b3_stock_info.parquet"

    # Converte DataFrame em parquet para memória
    buffer = BytesIO()
    
    df.to_parquet(buffer, engine="pyarrow",coerce_timestamps="us")
    #df.to_parquet(buffer, engine="pyarrow",compression="snappy")  
    #df.to_parquet(buffer, engine="pyarrow")

    # Cria cliente boto3 com credenciais explícitas
    s3 = boto3.client(
        "s3",
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        region_name="us-east-1"  # ajuste se necessário
    )

    # Faz upload para o S3
    s3.put_object(Bucket=BUCKET, Key=key, Body=buffer.getvalue())

    print(f"✅ Arquivo salvo em s3://{BUCKET}/{key}")


def get_b3_portfolio():
    # Lista de 10 tickers (ações brasileiras)
    tickers = [
        'PETR4.SA', 'VALE3.SA', 'ITUB4.SA', 'BBDC4.SA', 'ABEV3.SA',
        'BBAS3.SA', 'WEGE3.SA', 'ELET3.SA', 'SUZB3.SA', 'MGLU3.SA'
    ]

    # Data de ontem
    #ontem = datetime.today().date() - timedelta(days=1)

    # Baixar apenas o D-1
    #dados_intraday = yf.download(
    #    tickers,
    #    start=ontem,
    #    end=ontem + timedelta(days=1),  # precisa passar end = dia seguinte
    #    interval='5d'
    #)

    #return dados_intraday.reset_index()  # mantém Date como coluna
    
    dados_historico = yf.download(tickers, period='3d', interval='1d')
    
    df_pivot = dados_historico.stack(level=['Price', 'Ticker']).reset_index()

    # Renomeia colunas para ficar mais legível
    df_pivot.columns = ['Date', 'Status', 'Ticker', 'Value']
    df_pivot['Value'] = df_pivot['Value'].round(2)
    
    return df_pivot
    


if __name__ == "__main__":
    df = get_b3_portfolio()
    save_to_parquet(df)
    #print(df.columns)